In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/donors-choose/preprocessed_data.csv
/kaggle/input/glove6b100dtxt/glove.6B.100d.txt


In [0]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [0]:
embeddings_index = dict()
f = open('/kaggle/input/glove6b100dtxt/glove.6B.100d.txt',encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [0]:
data=pd.read_csv('/kaggle/input/donors-choose/preprocessed_data.csv',nrows=50000)

In [0]:
data.shape

(50000, 9)

In [0]:
data.columns

Index(['school_state', 'teacher_prefix', 'project_grade_category',
       'teacher_number_of_previously_posted_projects', 'project_is_approved',
       'clean_categories', 'clean_subcategories', 'essay', 'price'],
      dtype='object')

In [0]:
y=data['project_is_approved']

In [0]:
data=data.drop(['project_is_approved'],axis=1)

In [0]:
preproc=[]
for row in data['project_grade_category']:
    row=row.replace('grades','')
    row=row.replace('_prek_2','prek2')
    row=row.replace('_3_5','3to5')
    row=row.replace('_6_8','6to8')
    row=row.replace('_9_12','9to12')
    preproc.append(row)
    
data['project_grade_category']=preproc

In [0]:
preproc=[]
for row in data['clean_categories']:
    row=row.replace(' ','')
    row=row.replace('_','')
    preproc.append(row)
    
data['clean_categories']=preproc

In [0]:
preproc=[]
for row in data['clean_subcategories']:
    row=row.replace(' ','')
    row=row.replace('_','')
    preproc.append(row)
    
data['clean_subcategories']=preproc

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.33, stratify=y)
X_train, X_cv, y_train, y_cv = train_test_split(X_train, y_train, test_size=0.33, stratify=y_train)

In [0]:
X_train.columns

Index(['school_state', 'teacher_prefix', 'project_grade_category',
       'teacher_number_of_previously_posted_projects', 'clean_categories',
       'clean_subcategories', 'essay', 'price'],
      dtype='object')

In [0]:
from keras.models import Sequential,Model
from keras.layers import Dense,BatchNormalization
from keras.preprocessing.text import Tokenizer
from keras.layers import LSTM,Input,Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from keras.layers import BatchNormalization
from keras.initializers import he_normal

Using TensorFlow backend.


# Model 1

### Essay

In [0]:
#credit to https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/
token = Tokenizer()
token.fit_on_texts(X_train['essay'])
vocabulary_size = len(token.word_index)+1

In [0]:
embedding_matrix = np.zeros((vocabulary_size, 100))
for word, i in token.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [0]:
word_count = []

for e in data["essay"] :
    c = len(e.split())
    word_count.append(c)

In [0]:
max_word_count=max(word_count)

In [0]:
max_word_count

339

In [0]:
train_essay=token.texts_to_sequences(X_train['essay'])
test_essay=token.texts_to_sequences(X_test['essay'])
cv_essay=token.texts_to_sequences(X_cv['essay'])

In [0]:
train_essay=pad_sequences(train_essay,maxlen=350)
test_essay=pad_sequences(test_essay,maxlen=350)
cv_essay=pad_sequences(cv_essay,maxlen=350)

In [0]:
embedding_matrix.shape

(30527, 100)

In [0]:
max_length=350
input_essay = Input(shape=(max_length,))
embedding = Embedding(vocabulary_size, 100,weights=[embedding_matrix], input_length=max_length,trainable=False)(input_essay)
lstm=LSTM(100,return_sequences=True)(embedding)
flatten=Flatten()(lstm)

### School State

In [0]:
enc=LabelEncoder()
train_state=enc.fit_transform(X_train['school_state'])
test_state=enc.transform(X_test['school_state'])
cv_state=enc.transform(X_cv['school_state'])

In [0]:
vocabulary_size  = X_train["school_state"].nunique()
embedding_size_state= int(min(np.ceil((vocabulary_size)/2), 50 ))

In [0]:
input_state = Input(shape=(1,))
embedding=Embedding(vocabulary_size,embedding_size_state, input_length=1,trainable=False)(input_state)
flatten_1=Flatten()(embedding)

### Project Grade Category

In [0]:
enc=LabelEncoder()
train_grade=enc.fit_transform(X_train['project_grade_category'])
test_grade=enc.transform(X_test['project_grade_category'])
cv_grade=enc.transform(X_cv['project_grade_category'])

In [0]:
vocabulary_size  = X_train["project_grade_category"].nunique()
embedding_size_grade= int(min(np.ceil((vocabulary_size)/2), 50 ))

In [0]:
input_grade = Input(shape=(1,))
embedding=Embedding(vocabulary_size,embedding_size_grade, input_length=1,trainable=False)(input_grade)
flatten_2=Flatten()(embedding)

### Clean Categories

In [0]:
enc=LabelEncoder()
train_cat=enc.fit_transform(X_train['clean_categories'])
test_cat=enc.fit_transform(X_test['clean_categories'])
cv_cat=enc.fit_transform(X_cv['clean_categories'])

In [0]:
vocabulary_size  = X_train["clean_categories"].nunique()
embedding_size_cat= int(min(np.ceil((vocabulary_size)/2), 50 ))

In [0]:
input_cat = Input(shape=(1,))
embedding=Embedding(vocabulary_size,embedding_size_cat, input_length=1,trainable=False)(input_cat)
flatten_3=Flatten()(embedding)

### Clean Subcategories

In [0]:
enc=LabelEncoder()
train_sub=enc.fit_transform(X_train['clean_subcategories'])
test_sub=enc.fit_transform(X_test['clean_subcategories'])
cv_sub=enc.fit_transform(X_cv['clean_subcategories'])

In [0]:
vocabulary_size  = X_train["clean_subcategories"].nunique()
embedding_size_sub= int(min(np.ceil((vocabulary_size)/2), 50 ))

In [0]:
input_sub = Input(shape=(1,))
embedding=Embedding(vocabulary_size,embedding_size_sub, input_length=1,trainable=False)(input_sub)
flatten_4=Flatten()(embedding)

### Teacher Prefix

In [0]:
enc=LabelEncoder()
train_prefix=enc.fit_transform(X_train['teacher_prefix'])
test_prefix=enc.transform(X_test['teacher_prefix'])
cv_prefix=enc.transform(X_cv['teacher_prefix'])

In [0]:
vocabulary_size  = X_train["teacher_prefix"].nunique()
embedding_size_prefix= int(min(np.ceil((vocabulary_size)/2), 50 ))

In [0]:
input_prefix = Input(shape=(1,))
embedding=Embedding(vocabulary_size, embedding_size_prefix, input_length=1,trainable=False)(input_prefix)
flatten_5=Flatten()(embedding)

### Remaining Numerical Features

In [0]:
from scipy.sparse import hstack

train_price=X_train['price'].values.reshape(-1, 1)
train_number=X_train['teacher_number_of_previously_posted_projects'].values.reshape(-1, 1)

test_price=X_test['price'].values.reshape(-1, 1)
test_number=X_test['teacher_number_of_previously_posted_projects'].values.reshape(-1, 1)

cv_price=X_cv['price'].values.reshape(-1, 1)
cv_price=X_cv['teacher_number_of_previously_posted_projects'].values.reshape(-1, 1)

rem_train=np.concatenate((train_price,train_number),axis=1)
rem_test=np.concatenate((test_price,test_number),axis=1)
rem_cv=np.concatenate((cv_price,cv_price),axis=1)

In [0]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(rem_train)
rem_train_standard=scaler.transform(rem_train)
rem_test_standard=scaler.transform(rem_test)
rem_cv_standard=scaler.transform(rem_cv)

In [0]:
rem_feat = Input(shape=(2,))
rem_feat_dense= Dense(128, activation='relu')(rem_feat)

### Concatenating Data

In [0]:
train_data=[train_essay,np.array(train_state),np.array(train_grade),np.array(train_cat),np.array(train_sub),np.array(train_prefix),np.array(rem_train_standard)]
test_data=[test_essay,np.array(test_state),np.array(test_grade),np.array(test_cat),np.array(test_sub),np.array(test_prefix),np.array(rem_test_standard)]
cv_data=[cv_essay,np.array(cv_state),np.array(cv_grade),np.array(cv_cat),np.array(cv_sub),np.array(cv_prefix),np.array(rem_cv_standard)]

### Concatenating all the Layers

In [0]:
from keras.layers import concatenate
from keras.layers import Dropout
concat = concatenate([flatten,flatten_1,flatten_2,flatten_3,flatten_4,flatten_5,rem_feat_dense])

x = Dense(128, activation='relu',kernel_initializer=he_normal())(concat)
x=Dropout(0.6)(x)

x = Dense(256, activation='relu',kernel_initializer=he_normal())(x)
x=Dropout(0.6)(x)
x=BatchNormalization()(x)

x = Dense(64, activation='relu',kernel_initializer=he_normal())(x)

# And finally we add the main logistic regression layer
main_output = Dense(1, activation='sigmoid',kernel_initializer=he_normal())(x)

In [0]:
model1 = Model(inputs=[input_essay, input_state, input_grade, input_cat, input_sub, input_prefix,rem_feat], outputs=[main_output])

In [0]:
model1.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 350)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 350, 100)     3052700     input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_4 (I

In [0]:
# https://www.youtube.com/watch?v=2U6Jl7oqRkM
# https://www.tensorflow.org/tensorboard/r2/get_started

import tensorflow as tf
from keras.callbacks import TensorBoard,EarlyStopping
from time import time
earlystop_1 = EarlyStopping(monitor = 'val_loss', mode="min",min_delta = 0, patience = 5,verbose = 1,restore_best_weights = True)
tensorboard_1 = TensorBoard("logs1")

In [0]:
# https://github.com/pranaya-mathur/Donors-Choose-LSTM/blob/master/DonorsChoose_Model_1_13_Aug_19.ipynb
import tensorflow as tf
from sklearn.metrics import roc_auc_score

def auc_score(y_true, y_pred):
    if len(np.unique(y_true[:])) == 1:
        return 0.5
    else:
        return roc_auc_score(y_true, y_pred)

def auc_sc(y_true, y_pred):
    return tf.py_func(auc_score, (y_true, y_pred), tf.double)   

In [0]:
from keras.utils import np_utils
Y_train=np_utils.to_categorical(y_train)
Y_test=np_utils.to_categorical(y_test)
Y_cv=np_utils.to_categorical(y_cv)

In [0]:
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=[auc_sc])

In [0]:
model1.fit(train_data,y_train,batch_size=512, epochs=12,validation_data=(cv_data,y_cv),verbose=1,class_weight='balanced',callbacks=[tensorboard_1,earlystop_1])

Train on 22445 samples, validate on 11055 samples
Epoch 1/12
22445/22445 [==============================] - 27s 1ms/step - loss: 0.5433 - auc_sc: 0.5176 - val_loss: 0.4849 - val_auc_sc: 0.4917
Epoch 2/12
22445/22445 [==============================] - 25s 1ms/step - loss: 0.4514 - auc_sc: 0.5758 - val_loss: 0.4948 - val_auc_sc: 0.6761
Epoch 3/12
22445/22445 [==============================] - 25s 1ms/step - loss: 0.4348 - auc_sc: 0.6291 - val_loss: 0.4723 - val_auc_sc: 0.6977
Epoch 4/12
22445/22445 [==============================] - 25s 1ms/step - loss: 0.4236 - auc_sc: 0.6702 - val_loss: 0.5308 - val_auc_sc: 0.7056
Epoch 5/12
22445/22445 [==============================] - 25s 1ms/step - loss: 0.4191 - auc_sc: 0.6764 - val_loss: 0.5358 - val_auc_sc: 0.7161
Epoch 6/12
22445/22445 [==============================] - 26s 1ms/step - loss: 0.4135 - auc_sc: 0.6910 - val_loss: 0.4843 - val_auc_sc: 0.7188
Epoch 7/12
22445/22445 [==============================] - 25s 1ms/step - loss: 0.4108 - auc_

In [2]:
from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ["Model", "train_auc", "test_auc"]
x.add_row(["Model_1", 0.7100,0.7270])
print(x)

+---------+-----------+----------+
|  Model  | train_auc | test_auc |
+---------+-----------+----------+
| Model_1 |    0.71   |  0.727   |
+---------+-----------+----------+
